In [1]:
%cd '/content/drive/My Drive/Colab Notebooks/GraphNN'

/content/drive/My Drive/Colab Notebooks/GraphNN


In [2]:
import json
with open('result.json') as json_file:
    result_data = json.load(json_file)

with open('chair.json') as chair_file:
    chair_data = json.load(chair_file)

In [3]:
print(result_data)
print(chair_data)

[{'text': 'Sitting Furniture', 'children': [{'text': 'Chair Base', 'children': [{'text': 'Regular Leg Base', 'children': [{'text': 'Bar Stretcher', 'objs': ['new-0'], 'id': 12, 'name': 'bar_stretcher'}, {'text': 'Leg', 'objs': ['new-12'], 'id': 13, 'name': 'leg'}, {'text': 'Leg', 'objs': ['new-10'], 'id': 14, 'name': 'leg'}, {'text': 'Leg', 'objs': ['new-8'], 'id': 15, 'name': 'leg'}, {'text': 'Leg', 'objs': ['new-6'], 'id': 16, 'name': 'leg'}, {'text': 'Bar Stretcher', 'objs': ['new-4'], 'id': 17, 'name': 'bar_stretcher'}, {'text': 'Bar Stretcher', 'objs': ['new-2'], 'id': 18, 'name': 'bar_stretcher'}], 'id': 6, 'name': 'regular_leg_base'}], 'id': 5, 'name': 'chair_base'}, {'text': 'Chair Back', 'children': [{'text': 'Back Surface', 'children': [{'text': 'Back Hard Surface', 'objs': ['new-15'], 'id': 24, 'name': 'back_hard_surface'}], 'id': 23, 'name': 'back_surface'}], 'id': 19, 'name': 'chair_back'}, {'text': 'Chair Seat', 'children': [{'text': 'Seat Surface', 'children': [{'text': 

In [4]:
Layers = {}
import torch

def compute(root):
  for item in root:
    current = item['text']
    childrens = item.get('children')
    
    # AND or OR node
    if (childrens):
      
      # OR node
      if (len(childrens) == 1):
        print("OR Node")
        dict_entry = str(item['name']) + "_" + str(item['id'])
        print(dict_entry)
        compute(childrens)
      
      # AND node
      else:
        features = {}
        #features[current] = merge([compute(node) for node in childrens])

        #print(childrens)
        print("AND Node")
        dict_entry = str(item['name']) + "_" + str(item['id'])
        print(dict_entry)
        compute(childrens)

    # LEAF node
    else:
      print("LEAF Node")
      dict_entry = str(item['name']) + "_" + str(item['id']) 
      print(dict_entry)

def merge(features):
  return features.max(dim=0)

compute(result_data)

AND Node
chair_0
OR Node
chair_base_5
AND Node
regular_leg_base_6
LEAF Node
bar_stretcher_12
LEAF Node
leg_13
LEAF Node
leg_14
LEAF Node
leg_15
LEAF Node
leg_16
LEAF Node
bar_stretcher_17
LEAF Node
bar_stretcher_18
OR Node
chair_back_19
OR Node
back_surface_23
LEAF Node
back_hard_surface_24
OR Node
chair_seat_20
OR Node
seat_surface_21
LEAF Node
seat_hard_surface_22


In [5]:
AND_nodes = ["Sitting Furniture", "Chair Head", "Chair Back",
             "Chair Arm", "Chair Seat", "Regular Leg Base",
             "Star-shape Leg Base", "Mechanical Control", "Footrest Ring",
             "Pedestal Base", "Foot Base", "Surface Base", "Star-shape Leg Set",
             "Wheel Assembly"]
OR_nodes = ["Back Surface", "Back Frame", "Seat Surface", "Seat Frame", 
            "Footrest", "Chair Base"]
LEAF_nodes = ["Head Rest", "Head Connector", "Back Hard Surface",
              "Back Soft Surface", "Back Support", "Back Holistic Frame",
              "Back Surface Vertical Bar", "Back Surface Horizontal Bar",
              "Back Surface Slant Bar", "Back Frame Horizontal Bar", 
              "Back Frame Slant Bar", "Back Frame Vertical Bar", "Back Complex Decoration",
              "Back Connector", "Armrest Hard Surface", "Armrest Soft Surface",
              "Arm Horizontal Bar", "Arm Vertical Bar", "Arm Slant Bar", 
              "Arm Holistic Frame", "Sofa-style Arm", "Arm Writing Table",
              "Arm Connector", "Seat Hard Surface", "Seat Soft Surface",
              "Seat Support", "Seat Surface Vertical Bar",
              "Seat Surface Horizontal Bar", "Seat Surface Slant Bar",
              "Seat Frame Vertical Bar", "Seat Frame Horizontal Bar",
              "Seat Frame Slant Bar", "Seat Holistic Frame", "Pillow",
              "Footrest Connector", "Bar Stretcher", "Circular Stretcher",
              "Runner", "Rocker", "Seat Connector", "Central Support",
              "Lever", "Knob", "Button", "Mounting Plane", "Footrest Bar",
              "Footrest Circle", "Pedestal", "Short Leg", "Ground Surface",
              "Side Surface", "Leg", "Foot", "Brake", "Caster Yoke", "Caster Stem",
              "Mounting Plate", "Wheel"]

def parse_chair(chair):
  error_flag = False
  Tree_Layers = []
  Dest_Layers = []
  Stell_Src = []
  Stell_Dest = []
  def compute_tree(root):
    for item in root:
      current = item['text']
      #print(current)
      childrens = item.get('children')
      #print(childrens)
      # AND or OR node  
      if (childrens):
        source = str(item['name']) + "_" + str(item['id'])
        Tree_Layers.append(source)
        stell_source = str(item['text'])
        Stell_Src.append(stell_source)
        this_children = childrens
        Dest_Layers_current = []
        Stell_Dest_current = []
        for i in this_children:
          current_i = (str(i['name']) + "_" + str(i['id']))
          #print(current_i)
          Dest_Layers_current.append(current_i)
          Stell_Dest_current.append(str(i['text']))
        Dest_Layers.append(Dest_Layers_current)
        Stell_Dest.append(Stell_Dest_current)
        # AND node
        if (current in AND_nodes):
          #print("AND Node")
          dict_entry = str(item['name']) + "_" + str(item['id'])
          #print(dict_entry)
          compute_tree(childrens)

        elif (current in OR_nodes):
          #print("OR Node")
          dict_entry = str(item['name']) + "_" + str(item['id'])
          #print(dict_entry)
          compute_tree(childrens)

        else:
          error_flag = True
          return

      else:

        # LEAF node
        if (current in LEAF_nodes):
          #print("LEAF Node")
          dict_entry = str(item['name']) + "_" + str(item['id'])
          #print(dict_entry)
          #compute_tree(childrens)

        else:
          print("Error in classification.")
          return

  compute_tree(chair)
  if (error_flag == True):
    return [[],[]]

  print([Tree_Layers, Dest_Layers])
  return [Stell_Src, Stell_Dest]

print(parse_chair(result_data))

[['chair_0', 'chair_base_5', 'regular_leg_base_6', 'chair_back_19', 'back_surface_23', 'chair_seat_20', 'seat_surface_21'], [['chair_base_5', 'chair_back_19', 'chair_seat_20'], ['regular_leg_base_6'], ['bar_stretcher_12', 'leg_13', 'leg_14', 'leg_15', 'leg_16', 'bar_stretcher_17', 'bar_stretcher_18'], ['back_surface_23'], ['back_hard_surface_24'], ['seat_surface_21'], ['seat_hard_surface_22']]]
[['Sitting Furniture', 'Chair Base', 'Regular Leg Base', 'Chair Back', 'Back Surface', 'Chair Seat', 'Seat Surface'], [['Chair Base', 'Chair Back', 'Chair Seat'], ['Regular Leg Base'], ['Bar Stretcher', 'Leg', 'Leg', 'Leg', 'Leg', 'Bar Stretcher', 'Bar Stretcher'], ['Back Surface'], ['Back Hard Surface'], ['Seat Surface'], ['Seat Hard Surface']]]


In [7]:
import pickle 
sit_data = pickle.load(open("sit_data_2.pkl", "rb"))

In [9]:
all_chair = []
chair_folder = '/content/drive/My Drive/Colab Notebooks/GraphNN/OBB/Chair'
import os
for d in os.listdir(chair_folder):
  all_chair.append(d[:-4])

print(all_chair)

['313C1FCC', '2D2C7514', 'DDCDEB1B', '02D23CEC', 'A55359B8', 'D1368CC9', 'BD564507', '92F001A2', '8F33260A', '1F550C17', '08C198E9', '90B5D32C', 'F5C05823', 'EE724C57', 'BDE777B6', '47571A16', 'F97B5A55', 'D3C6D323', '96FF362A', '32ECE3BC', '1FF3CC2E', '.ipynb_checkpo']


In [10]:
annotations_folder = '/content/drive/My Drive/Colab Notebooks/GraphNN/annotations'
folders = []
models = []
model_folder = []
for d in os.listdir(annotations_folder):
  for x in os.listdir('/content/drive/My Drive/Colab Notebooks/GraphNN/annotations/{}'.format(d)):
    current_dir = '/content/drive/My Drive/Colab Notebooks/GraphNN/annotations/{}'.format(d)
    if (x == "meta.json"):
      current_file = '{}/meta.json'.format(current_dir)
      with open(current_file) as to_parse:
        json_output = json.load(to_parse)
        model = json_output['model_id']
        if (model in all_chair):
          models.append(model)
          folders.append(d)
          model_folder.append([model,d])

print(folders)
print(len(folders))
from collections import Counter
print(len(models))
print(Counter(models))
print(model_folder)

['90_1', '98_1', '97_1', '94_1', '89_1', '92_1', '99_1', '91_1', '67_3', '68_2', '68_1', '67_2', '66_1', '113_1', '112_1', '111_1', '109_1', '111_2', '110_2', '110_1', '104_2', '108_1', '105_1', '101_1', '104_1', '100_1']
26
26
Counter({'08C198E9': 2, '1F550C17': 2, 'EE724C57': 2, 'DDCDEB1B': 2, 'BD564507': 2, '2D2C7514': 1, '90B5D32C': 1, '8F33260A': 1, '47571A16': 1, '1FF3CC2E': 1, '32ECE3BC': 1, '92F001A2': 1, '313C1FCC': 1, '02D23CEC': 1, 'F97B5A55': 1, 'F5C05823': 1, 'D3C6D323': 1, 'D1368CC9': 1, 'BDE777B6': 1, 'A55359B8': 1, '96FF362A': 1})
[['2D2C7514', '90_1'], ['90B5D32C', '98_1'], ['8F33260A', '97_1'], ['47571A16', '94_1'], ['1FF3CC2E', '89_1'], ['32ECE3BC', '92_1'], ['92F001A2', '99_1'], ['313C1FCC', '91_1'], ['08C198E9', '67_3'], ['1F550C17', '68_2'], ['1F550C17', '68_1'], ['08C198E9', '67_2'], ['02D23CEC', '66_1'], ['F97B5A55', '113_1'], ['F5C05823', '112_1'], ['EE724C57', '111_1'], ['D3C6D323', '109_1'], ['EE724C57', '111_2'], ['DDCDEB1B', '110_2'], ['DDCDEB1B', '110_1'],

In [11]:
json_to_parse = []
for d in os.listdir(annotations_folder):
  if (d not in folders):
    continue
  else:
    for x in os.listdir('/content/drive/My Drive/Colab Notebooks/GraphNN/annotations/{}'.format(d)):
      current_dir = '/content/drive/My Drive/Colab Notebooks/GraphNN/annotations/{}'.format(d)
      if (x == "result.json"):
        current_file = '{}/result.json'.format(current_dir)
        with open(current_file) as to_parse:
          json_output = json.load(to_parse)
          json_to_parse.append(json_output)

print(len(json_to_parse))
print(json_to_parse)

26
[[{'text': 'Sitting Furniture', 'children': [{'text': 'Chair Back', 'children': [{'text': 'Back Surface', 'children': [{'text': 'Back Hard Surface', 'objs': ['new-15'], 'id': 18, 'name': 'back_hard_surface'}], 'id': 17, 'name': 'back_surface'}], 'id': 1, 'name': 'chair_back'}, {'text': 'Chair Arm', 'children': [{'text': 'Armrest Hard Surface', 'objs': ['new-17'], 'id': 15, 'name': 'armrest_hard_surface'}, {'text': 'Arm Vertical Bar', 'objs': ['new-16'], 'id': 16, 'name': 'arm_vertical_bar'}], 'id': 2, 'name': 'chair_arm'}, {'text': 'Chair Arm', 'children': [{'text': 'Armrest Hard Surface', 'objs': ['new-13'], 'id': 13, 'name': 'armrest_hard_surface'}, {'text': 'Arm Vertical Bar', 'objs': ['new-12'], 'id': 14, 'name': 'arm_vertical_bar'}], 'id': 3, 'name': 'chair_arm'}, {'text': 'Chair Seat', 'children': [{'text': 'Seat Surface', 'children': [{'text': 'Seat Hard Surface', 'objs': ['new-8'], 'id': 12, 'name': 'seat_hard_surface'}], 'id': 11, 'name': 'seat_surface'}], 'id': 4, 'name': 

In [41]:
#!pip install trimesh
#!pip install pyquaternion
c_pkl_data = pickle.load(open("/content/drive/My Drive/Colab Notebooks/GraphNN/OBB/Chair/F97B5A55.pkl","rb"))
for k,v in c_pkl_data[0].items():
  print(k)
  print(v)

Back Hard Surface
[(TrackedArray(-0.22235288), TrackedArray(0.47949534), TrackedArray(-0.28564042), TrackedArray(0.62851395), TrackedArray(0.63556306), TrackedArray(0.12173586), Quaternion(0.748234172614717, 0.6632350573743163, 0.012189542928980565, 0.010784092193579798))]
Back Surface
[None]
Chair Back
[None]
Armrest Hard Surface
[(TrackedArray(-0.71024695), TrackedArray(0.2881826), TrackedArray(-0.19382198), TrackedArray(0.35744513), TrackedArray(0.06828202), TrackedArray(0.04478346), Quaternion(0.01193119781422015, 0.6786931791171389, 0.7342605883094432, -0.009726437891642492)), (TrackedArray(-0.71275906), TrackedArray(-0.30068381), TrackedArray(-0.1875463), TrackedArray(0.35521816), TrackedArray(0.0709693), TrackedArray(0.04474545), Quaternion(0.011737760427826874, 0.733020031394292, 0.6800492661547317, -0.008766650287394753))]
Chair Arm
[None, None]
Seat Hard Surface
[(TrackedArray(-0.28727664), TrackedArray(0.24292302), TrackedArray(-0.20062976), TrackedArray(0.60243487), Tracked

In [17]:
print(len(sit_data))
index = 0
hashes = []
hashx = []
for sit in sit_data:
  person = sit['object']
  hash1 = person['hash']
  if (hash1 in all_chair):
    hashx.append(hash1)
    hashes.append(index)
  index = index + 1

print(hashes)
print(len(hashes))
print(hashx)
print(len(hashx))

179
[2, 4, 6, 7, 14, 15, 16, 17, 18, 20, 21, 22, 23, 26, 29, 30, 33, 35, 36, 37, 39, 40, 41, 42, 44, 45, 46, 51, 61, 63, 64, 67, 68, 69, 79, 80, 81, 82, 83, 84, 85, 86, 87, 96, 97, 98, 99, 100, 101, 102, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 172, 173, 174, 175, 176, 177, 178]
121
['02D23CEC', '02D23CEC', '08C198E9', '08C198E9', 'DDCDEB1B', 'A55359B8', '96FF362A', '96FF362A', '96FF362A', '96FF362A', '96FF362A', '08C198E9', '08C198E9', '08C198E9', '313C1FCC', '08C198E9', 'D3C6D323', '313C1FCC', 'D3C6D323', '08C198E9', '08C198E9', '08C198E9', '313C1FCC', '08C198E9', 'D3C6D323', 'D3C6D323', 'D3C6D323', '1F550C17', '08C198E9', '32ECE3BC', 'BDE777B6', 'BD564507', 'EE724C57', 'EE724C57', '1FF3CC2E', '1FF3CC2E', '1FF

In [18]:
def intersection(lst1, lst2): 
    lst3 = [value for value in lst1 if value in lst2] 
    return lst3

print(intersection(all_chair, hashx))
print(len(intersection(all_chair,hashx)))

['313C1FCC', '2D2C7514', 'DDCDEB1B', '02D23CEC', 'A55359B8', 'D1368CC9', 'BD564507', '92F001A2', '8F33260A', '1F550C17', '08C198E9', '90B5D32C', 'F5C05823', 'EE724C57', 'BDE777B6', '47571A16', 'F97B5A55', 'D3C6D323', '96FF362A', '32ECE3BC', '1FF3CC2E']
21


In [ ]:
import os
bbox_dir = os.path.join('OBB')
bbox_dict = {}
for d in os.listdir(bbox_dir):
    if '.' in d or '_' in d:
        continue
    for f in os.listdir(os.path.join(bbox_dir, d)):
        print(f)
        if len(f) > 12:
            continue
        bboxes = pickle.load(open(os.path.join(bbox_dir, d, f), 'rb'))[0]
        bbox_dict[f[:-4]] = bboxes

In [20]:
#!pip install trimesh
#!pip install pyquaternion
chair_pkl_data = pickle.load(open("/content/drive/My Drive/Colab Notebooks/GraphNN/OBB/Chair/F97B5A55.pkl","rb"))

In [21]:
print(chair_pkl_data)

[defaultdict(<class 'list'>, {'Back Hard Surface': [(TrackedArray(-0.22235288), TrackedArray(0.47949534), TrackedArray(-0.28564042), TrackedArray(0.62851395), TrackedArray(0.63556306), TrackedArray(0.12173586), Quaternion(0.748234172614717, 0.6632350573743163, 0.012189542928980565, 0.010784092193579798))], 'Back Surface': [None], 'Chair Back': [None], 'Armrest Hard Surface': [(TrackedArray(-0.71024695), TrackedArray(0.2881826), TrackedArray(-0.19382198), TrackedArray(0.35744513), TrackedArray(0.06828202), TrackedArray(0.04478346), Quaternion(0.01193119781422015, 0.6786931791171389, 0.7342605883094432, -0.009726437891642492)), (TrackedArray(-0.71275906), TrackedArray(-0.30068381), TrackedArray(-0.1875463), TrackedArray(0.35521816), TrackedArray(0.0709693), TrackedArray(0.04474545), Quaternion(0.011737760427826874, 0.733020031394292, 0.6800492661547317, -0.008766650287394753))], 'Chair Arm': [None, None], 'Seat Hard Surface': [(TrackedArray(-0.28727664), TrackedArray(0.24292302), Tracked

In [ ]:
from google.colab import files
def getLocalFiles():
    _files = files.upload()
    if len(_files) >0:
       for k,v in _files.items():
         open(k,'wb').write(v)
getLocalFiles()

Saving kinematics.py to kinematics.py
Saving prepare_gt_data.py to prepare_gt_data.py
Saving visualization.py to visualization.py
Saving A-shape.txt to A-shape.txt


In [51]:
hoi_data = pickle.load(open('sit_data_2.pkl', 'rb'))
total = len(hoi_data)
print(total)
print(hoi_data[178])

179
{'person': {'id': '000574465', 'hash': 'F6D1E04E', 'root_position': ['-1008.556885', '-475.584930', '49.597954'], 'root_rotation': ['0.000000', '0.000000', '0.998889', '-0.047118'], 'skeleton': {'SKEL_ROOT': ['-1008.561646', '-475.573578', '49.582195'], 'SKEL_Pelvis': ['-1008.561646', '-475.573578', '49.582195'], 'SKEL_L_Thigh': ['-1008.486755', '-475.547607', '49.490768'], 'SKEL_L_Calf': ['-1008.306885', '-475.912140', '49.493202'], 'SKEL_L_Foot': ['-1008.494934', '-476.015747', '49.140415'], 'SKEL_L_Toe0': ['-1008.407471', '-476.142334', '49.091175'], 'IK_L_Foot': ['-1008.450928', '-476.079346', '49.048397'], 'PH_L_Foot': ['-1008.450928', '-476.079346', '49.048397'], 'MH_L_Knee': ['-1008.309814', '-475.906311', '49.493160'], 'SKEL_R_Thigh': ['-1008.672913', '-475.565216', '49.535400'], 'SKEL_R_Calf': ['-1008.827393', '-475.941040', '49.546734'], 'SKEL_R_Foot': ['-1008.825317', '-475.866516', '49.140553'], 'SKEL_R_Toe0': ['-1008.929138', '-475.979462', '49.090080'], 'IK_R_Foot': [

In [73]:
import os
import numpy as np
from pyquaternion.quaternion import Quaternion as Q
import pickle
import torch
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import kinematics as K
from visualization import draw_obj

for i_data, data_dict in enumerate(hoi_data):
    if i_data in [65, 67, 78, 97, 103, 131, 144, 149, 150, 151, 152, 153, 160, 173, 177, 178]:   # incorrect sitting
        continue
    if i_data in [14, 68, 69, 70, 71, 129, 130, 131, 136, 137, 138, 139, 140, 141, 169]:   # incorrect labeling -> incorrect relationship 130? 169?
        continue
    #print('\r%d/%d'%(i_data, total), end='')
    if (i_data == 176):
      print(i_data)
      # extract obj bboxes
      obj = data_dict['object']
      if obj['hash'] not in bbox_dict:
          continue
      # extract skeleton
      skeleton = data_dict['person']['skeleton']
      print(skeleton)
      skeleton = np.array([skeleton[x] for x in K.joint_names]).astype(np.float32)
      center = np.array(data_dict['person']['root_position']).astype(np.float32)
      skeleton -= center
      print(skeleton)
      rotation = np.array(data_dict['person']['root_rotation']).astype(np.float32)[[3,0,1,2]]
      skeleton = np.matmul(Q(rotation).inverse.rotation_matrix, skeleton.T).T
      print(skeleton)
      obj_config = []
      obj_cat_id = []
      for part_id in bbox_dict[obj['hash']]:
          print(part_id)
          for bbox in bbox_dict[obj['hash']][part_id]:
              print(bbox)
              if bbox is None:
                  continue
              T1,T2,T3,D,W,H,R = bbox
              R2 = Q(rotation).inverse * Q(np.array(obj['root_rotation'])[[3,0,1,2]])
              dx, dy, dz = (float(x) for x in obj['root_position'])
              dx -= center[0]
              dy -= center[1]
              dz -= center[2]
              obj_config.append([T3,T2,T1,D,W,H,*R.rotation_matrix[:,:2].reshape([6]),dx,dy,dz,*R2.rotation_matrix[:,:2].reshape([6])])
              obj_cat_id.append(part_id)

176
{'SKEL_ROOT': ['-1007.894592', '-477.781830', '49.627159'], 'SKEL_Pelvis': ['-1007.894592', '-477.781830', '49.627159'], 'SKEL_L_Thigh': ['-1007.987061', '-477.797119', '49.550617'], 'SKEL_L_Calf': ['-1008.293518', '-477.530731', '49.530907'], 'SKEL_L_Foot': ['-1008.191956', '-477.599121', '49.136517'], 'SKEL_L_Toe0': ['-1008.292542', '-477.487457', '49.077435'], 'IK_L_Foot': ['-1008.242371', '-477.548187', '49.039742'], 'PH_L_Foot': ['-1008.242371', '-477.548187', '49.039742'], 'MH_L_Knee': ['-1007.896423', '-477.780182', '49.647018'], 'SKEL_R_Thigh': ['-1007.807373', '-477.729645', '49.561504'], 'SKEL_R_Calf': ['-1007.741577', '-477.329041', '49.540211'], 'SKEL_R_Foot': ['-1007.791321', '-477.426575', '49.142017'], 'SKEL_R_Toe0': ['-1007.773315', '-477.277222', '49.083145'], 'IK_R_Foot': ['-1007.779907', '-477.355682', '49.045319'], 'PH_R_Foot': ['-1007.779907', '-477.355682', '49.045319'], 'MH_R_Knee': ['-1007.896423', '-477.780182', '49.647018'], 'RB_L_ThighRoll': ['-1007.98706

In [29]:

for i in range(len(obj_config)):
  print(len(obj_config[i]))
  print(obj_config[i])
  print(obj_cat_id[i])

21
[TrackedArray(-0.36515037), TrackedArray(-0.09992039), TrackedArray(0.46116127), TrackedArray(0.71089036), TrackedArray(0.23980003), TrackedArray(0.48941606), 0.9996479519050888, 0.022261587788203287, -0.017806720203175523, 0.9662508226280256, -0.019669595002602114, 0.2566393763236871, 0.07781933984369971, -0.18249511523436013, -0.6179194326171853, 0.9998093279282712, -0.014411497582678914, 0.014121163563842824, 0.9996611305537306, -0.013487050352883674, -0.021677933422287226]
Back Hard Surface
21
[TrackedArray(-0.09887844), TrackedArray(-0.29865476), TrackedArray(-0.76939002), TrackedArray(0.40874203), TrackedArray(0.08977465), TrackedArray(0.08694582), -0.06454930224451848, 0.9941532697263556, 0.9969823711678015, 0.0679942388045012, -0.04312237424323685, 0.0838812242506767, 0.07781933984369971, -0.18249511523436013, -0.6179194326171853, 0.9998093279282712, -0.014411497582678914, 0.014121163563842824, 0.9996611305537306, -0.013487050352883674, -0.021677933422287226]
Armrest Hard Su